In [116]:
import os
import pandas as pd
import numpy as np
import pickle
import gc

TEST_CSV = r'C:\Users\e0817820\Desktop\tokka\data\raw\final_test.csv'
ADDITIONAL_TRAIN_CSV = r'C:\Users\e0817820\Desktop\tokka\data\additional_lastpoint\add_lastpoint.csv'
MODELS_DIR = r'C:\Users\e0817820\Desktop\tokka\models'

In [117]:
list_cryptocurrencies = ['AVAX', 'ADA', 'SOL', 'BNB', 'TRX', 'DOGE', 'LINK', 'XRP', 'BTC', 'ETH']

# df_train = pd.read_csv(ADDITIONAL_TRAIN_CSV)
df_test = pd.read_csv(TEST_CSV)
sample = df_test[['log_return']]
real_target = []
for symbol in list_cryptocurrencies:
    real_target.append(df_test[df_test['symbol'] == symbol]['log_return'])
df_test = df_test.drop(columns=['log_return'])
lower_limit_test = df_test['timestamp'].min()
upper_limit_train = lower_limit_test-400*60*1000-1
lower_limit_test_datetime = pd.to_datetime(lower_limit_test, unit='ms')
upper_limit_train_datetime = pd.to_datetime(upper_limit_train, unit='ms')
# df_train.drop(columns=['Unnamed: 0'])

In [118]:
list_cols = ['timestamp', 'open', 'high', 'low', 'close', 'volume',
       'quote_asset_volume', 'number_of_trades', 'taker_buy_volume',
       'taker_sell_volume', 'symbol']

In [119]:
# df_combined = pd.concat([df_train, df_test])
df_combined = df_test.copy()
df_combined['timestamp'] = pd.to_datetime(df_combined['timestamp'], unit='ms')
all_timestamps = pd.date_range(start=upper_limit_train_datetime, end=lower_limit_test_datetime, freq='T')  # Adjust the freq='T' as necessary

expanded_df = pd.DataFrame()
for symbol in list_cryptocurrencies:
    temp_df = pd.DataFrame({'timestamp': all_timestamps})
    temp_df['symbol'] = symbol 
    temp_df = pd.merge(temp_df, df_combined[df_combined['symbol'] == symbol], on='timestamp', how='left', suffixes=('', '_drop'))
    temp_df.drop([col for col in temp_df if col.endswith('_drop')], axis=1, inplace=True)
    expanded_df = pd.concat([expanded_df, temp_df])

expanded_df = expanded_df.sort_values(by=['timestamp', 'symbol'], ascending=[True, True])
expanded_df['timestamp'] = (expanded_df['timestamp'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')
df_filled = pd.concat([expanded_df, df_test])
df_filled = df_filled.groupby('symbol').apply(lambda group: group.bfill())
df_filled.reset_index(drop=True, inplace=True)
del df_combined

df_filled.rename(columns={'index': 'timestamp'}, inplace=True)
print(df_filled.columns)
# df_filled.drop(columns=['id'], inplace=True)

Index(['timestamp', 'symbol', 'id', 'open', 'high', 'low', 'close', 'volume',
       'quote_asset_volume', 'number_of_trades', 'taker_buy_volume',
       'taker_sell_volume'],
      dtype='object')


C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\3701895130.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  all_timestamps = pd.date_range(start=upper_limit_train_datetime, end=lower_limit_test_datetime, freq='T')  # Adjust the freq='T' as necessary
C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\3701895130.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filled = df_filled.groupby('symbol').apply(lambda group: group.bfill())


In [120]:
print(df_filled.shape)

(148020, 12)


In [121]:
df_filled.to_csv(r'C:\Users\e0817820\Desktop\tokka\data\raw\test_filled.csv', index=False)

In [122]:
df_filled.head()

,timestamp,symbol,id,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_volume,taker_sell_volume
0,1709198399999,ADA,4.0,0.6928,0.6955,0.6928,0.6954,380367.0,264019.16844,352.0,306265.4,212567.9046
1,1709198459999,ADA,4.0,0.6928,0.6955,0.6928,0.6954,380367.0,264019.16844,352.0,306265.4,212567.9046
2,1709198519999,ADA,4.0,0.6928,0.6955,0.6928,0.6954,380367.0,264019.16844,352.0,306265.4,212567.9046
3,1709198579999,ADA,4.0,0.6928,0.6955,0.6928,0.6954,380367.0,264019.16844,352.0,306265.4,212567.9046
4,1709198639999,ADA,4.0,0.6928,0.6955,0.6928,0.6954,380367.0,264019.16844,352.0,306265.4,212567.9046


In [123]:
combined_df1 = pd.DataFrame()
combined_df1[df_filled.columns] = 0
for id in range(10):
    combined_df1 = combined_df1.merge(df_filled.loc[df_filled["symbol"] == list_cryptocurrencies[id], ['timestamp', 'open', 'high', 'low', 'close', 'volume']].copy(), on="timestamp", how='outer', suffixes=['', "_"+list_cryptocurrencies[id]])
print(combined_df1.isna().sum())
combined_df1 = combined_df1.drop(df_filled.columns.drop("timestamp"), axis=1)
combined_df1 = combined_df1.sort_values('timestamp', ascending=True)
combined_df1 = combined_df1.fillna(method='ffill')
combined_df1.drop_duplicates(subset=['timestamp'], inplace=True)

combined_df1 = combined_df1[combined_df1['timestamp'] > lower_limit_test - 400*60*1000]
display(combined_df1.head())

timestamp         0
symbol        14802
id            14802
open          14802
high          14802
              ...  
open_ETH          0
high_ETH          0
low_ETH           0
close_ETH         0
volume_ETH        0
Length: 62, dtype: int64


C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\1470829341.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df1 = combined_df1.fillna(method='ffill')


,timestamp,open_AVAX,high_AVAX,low_AVAX,close_AVAX,volume_AVAX,open_ADA,high_ADA,low_ADA,close_ADA,...,open_BTC,high_BTC,low_BTC,close_BTC,volume_BTC,open_ETH,high_ETH,low_ETH,close_ETH,volume_ETH
1,1709198459999,43.51,43.63,43.51,43.61,6615.2,0.6928,0.6955,0.6928,0.6954,...,62768.01,62860.0,62753.92,62853.74,64.93466,3468.42,3470.6,3468.06,3469.82,758.9921
2,1709198519999,43.51,43.63,43.51,43.61,6615.2,0.6928,0.6955,0.6928,0.6954,...,62768.01,62860.0,62753.92,62853.74,64.93466,3468.42,3470.6,3468.06,3469.82,758.9921
3,1709198579999,43.51,43.63,43.51,43.61,6615.2,0.6928,0.6955,0.6928,0.6954,...,62768.01,62860.0,62753.92,62853.74,64.93466,3468.42,3470.6,3468.06,3469.82,758.9921
4,1709198639999,43.51,43.63,43.51,43.61,6615.2,0.6928,0.6955,0.6928,0.6954,...,62768.01,62860.0,62753.92,62853.74,64.93466,3468.42,3470.6,3468.06,3469.82,758.9921
5,1709198699999,43.51,43.63,43.51,43.61,6615.2,0.6928,0.6955,0.6928,0.6954,...,62768.01,62860.0,62753.92,62853.74,64.93466,3468.42,3470.6,3468.06,3469.82,758.9921


In [124]:
combined_df2 = pd.DataFrame()
combined_df2[df_filled.columns] = 0
for id in range(10):
    combined_df2 = combined_df2.merge(df_filled.loc[df_filled["symbol"] == list_cryptocurrencies[id], ['timestamp', 'open', 'high', 'low', 'close', 'volume']].copy(), on="timestamp", how='outer', suffixes=['', "_"+list_cryptocurrencies[id]])
combined_df2 = combined_df2.drop(df_filled.columns.drop("timestamp"), axis=1)
combined_df2 = combined_df2.sort_values('timestamp', ascending=True)
combined_df2 = combined_df2.fillna(method='ffill')
combined_df2.drop_duplicates(subset=['timestamp'], inplace=True)

combined_df2 = combined_df2[combined_df2['timestamp'] > lower_limit_test - 400*60*1000]
display(combined_df2.head())

C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\628046445.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df2 = combined_df2.fillna(method='ffill')


,timestamp,open_AVAX,high_AVAX,low_AVAX,close_AVAX,volume_AVAX,open_ADA,high_ADA,low_ADA,close_ADA,...,open_BTC,high_BTC,low_BTC,close_BTC,volume_BTC,open_ETH,high_ETH,low_ETH,close_ETH,volume_ETH
1,1709198459999,43.51,43.63,43.51,43.61,6615.2,0.6928,0.6955,0.6928,0.6954,...,62768.01,62860.0,62753.92,62853.74,64.93466,3468.42,3470.6,3468.06,3469.82,758.9921
2,1709198519999,43.51,43.63,43.51,43.61,6615.2,0.6928,0.6955,0.6928,0.6954,...,62768.01,62860.0,62753.92,62853.74,64.93466,3468.42,3470.6,3468.06,3469.82,758.9921
3,1709198579999,43.51,43.63,43.51,43.61,6615.2,0.6928,0.6955,0.6928,0.6954,...,62768.01,62860.0,62753.92,62853.74,64.93466,3468.42,3470.6,3468.06,3469.82,758.9921
4,1709198639999,43.51,43.63,43.51,43.61,6615.2,0.6928,0.6955,0.6928,0.6954,...,62768.01,62860.0,62753.92,62853.74,64.93466,3468.42,3470.6,3468.06,3469.82,758.9921
5,1709198699999,43.51,43.63,43.51,43.61,6615.2,0.6928,0.6955,0.6928,0.6954,...,62768.01,62860.0,62753.92,62853.74,64.93466,3468.42,3470.6,3468.06,3469.82,758.9921


In [125]:
print(combined_df1.shape)
print(combined_df2.shape)

(14801, 51)
(14801, 51)


In [126]:
def bollinger_mavg(close, window=20):
    """Calculate the middle Bollinger Band."""
    return close.rolling(window=window).mean()

def bollinger_hband(close, window=20, k=2):
    """Calculate the upper Bollinger Band."""
    mavg = bollinger_mavg(close, window)
    mstd = close.rolling(window=window).std()
    return mavg + (mstd * k)

def bollinger_lband(close, window=20, k=2):
    """Calculate the lower Bollinger Band."""
    mavg = bollinger_mavg(close, window)
    mstd = close.rolling(window=window).std()
    return mavg - (mstd * k)

def bollinger_wband(close, window=20, k=2):
    """Calculate the width of the Bollinger Bands."""
    hband = bollinger_hband(close, window, k)
    lband = bollinger_lband(close, window, k)
    mavg = bollinger_mavg(close, window)
    return ((hband - lband) / mavg) * 100

def exponential_moving_average(close, span=20):
    """Calculate the Exponential Moving Average (EMA)."""
    return close.ewm(span=span, adjust=False).mean()

def relative_strength_index(close, window=14):
    """Calculate the Relative Strength Index (RSI)."""
    delta = close.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi
  
def stochastic_oscillator(high, low, close, window=14):
    """Calculate the Stochastic Oscillator."""
    low_min = low.rolling(window=window).min()
    high_max = high.rolling(window=window).max()
    k = ((close - low_min) / (high_max - low_min)) * 100
    return k

def williams_r(high, low, close, window=14):
    """Calculate Williams %R."""
    high_max = high.rolling(window=window).max()
    low_min = low.rolling(window=window).min()
    r = -100 * ((high_max - close) / (high_max - low_min))
    return r
  
def rate_of_change(close, window=14):
    """Calculate the Rate of Change (ROC)."""
    roc = ((close - close.shift(window)) / close.shift(window)) * 100
    return roc

In [127]:
def delay(x, d):
    """Value of x d days ago"""
    return x.shift(d)

def correlation(x, y, d):
    """Time-series correlation of x and y for the past d days"""
    return x.rolling(window=d).corr(y)

def delta(x, d):
    """Today's value of x minus the value of x d days ago"""
    return x.diff(d)

def ts_max(x, d):
    """Time-series max over the past d days"""
    return x.rolling(window=d).max()

def ts_min(x, d):
    """Time-series min over the past d days"""
    return x.rolling(window=d).min()

In [128]:
def getFeaturesVer1(df, limit):
    lags_normal = [15, 40, 150, 300]
    lags_ema = [15, 50, 160, 300]
    lags_rsi = [15, 50, 160, 300]
    lags_bbands = [15, 50, 160, 300]
    for id in range(10):   
        symbol = list_cryptocurrencies[id] 
        for lag in lags_normal:
            df[f'log_close/mean_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.append(np.convolve( np.array(df[f'close_{symbol}']), np.ones(lag)/lag, mode="valid"), np.ones(lag-1)), lag-1)  )
            df[f'log_return_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.array(df[f'close_{symbol}']), lag)  )
    for lag in lags_normal:
        df[f'mean_close/mean_{lag}'] =  np.mean(df.iloc[:,df.columns.str.startswith(f'log_close/mean_{lag}_')], axis=1)
        df[f'mean_log_returns_{lag}'] = np.mean(df.iloc[:,df.columns.str.startswith(f'log_return_{lag}_')], axis=1)
        for id in range(10):
            symbol = list_cryptocurrencies[id] 
            df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
            df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
    # apply TA indicators: EMA, RSI, Bollinger Bands, Stochastic Oscillator
    for id in range(10):
        symbol = list_cryptocurrencies[id]
        for lag in lags_ema:
            # df[f'ema_close_{lag}_{symbol}'] = ta.trend.ema_indicator(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'ema_close_{lag}_{symbol}'] = exponential_moving_average(df[f'close_{symbol}'], span=lag)
        for lag in lags_rsi:
            # df[f'rsi_close_{lag}_{symbol}'] = ta.momentum.rsi(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'rsi_close_{lag}_{symbol}'] = relative_strength_index(df[f'close_{symbol}'], window=lag)
        for lag in lags_bbands:
            # df[f'hbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_hband(df[f'close_{symbol}'], window=lag, fillna=True)
            # df[f'lbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_lband(df[f'close_{symbol}'], window=lag, fillna=True)
            # df[f'width_bbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_wband(df[f'close_{symbol}'], window=lag, fillna=True)
            df[f'hbbands_close_{lag}_{symbol}'] = bollinger_hband(df[f'close_{symbol}'], window=lag)
            df[f'lbbands_close_{lag}_{symbol}'] = bollinger_lband(df[f'close_{symbol}'], window=lag)
            df[f'width_bbands_close_{lag}_{symbol}'] = bollinger_wband(df[f'close_{symbol}'], window=lag)
        # Alpha#12: (sign(delta(volume, 1)) * (-1 * delta(close, 1))) 
        df['alpha12_'+symbol] = np.sign(delta(df['volume_'+symbol], 1)) * (-1 * delta(df['close_'+symbol], 1))
        # Alpha#54: ((-1 * ((low - close) * (open^5))) / (0.0000001 + (low - high) * (close^5))) 
        df['alpha54_'+symbol] = ((-1 * ((df['low_'+symbol] - df['close_'+symbol]) * (df['open_'+symbol]**5))) / (0.0000001 + (df['low_'+symbol] - df['high_'+symbol]) * (df['close_'+symbol]**5)))
        # Alpha#41: (((high * low)^0.5) - close)
        # df['alpha41_'+symbol] = ((df['high_'+symbol] * df['low_'+symbol])**0.5) - df['close_'+symbol]
        # Alpha#53: (-1 * delta((((close - low) - (high - close)) / (close - low + 0.0000001)), 9))
        # df['alpha53_'+symbol] = -1 * delta((((df['close_'+symbol] - df['low_'+symbol]) - (df['high_'+symbol] - df['close_'+symbol])) / (df['close_'+symbol] - df['low_'+symbol] + 0.0000001)), 9)
        # Alpha#6: (-1 * correlation(open, volume, 10)) 
        # df['alpha6_'+symbol] = -1 * correlation(df['open_'+symbol], df['volume_'+symbol], 10)
        # Alpha#9: ((0 < ts_min(delta(close, 1), 5)) ? delta(close, 1) : ((ts_max(delta(close, 1), 5) < 0) ? delta(close, 1) : (-1 * delta(close, 1)))) 
        df['alpha9_'+symbol] = np.where(0 < ts_min(delta(df['close_'+symbol], 1), 5), delta(df['close_'+symbol], 1), np.where(ts_max(delta(df['close_'+symbol], 1), 5) < 0, delta(df['close_'+symbol], 1), -1 * delta(df['close_'+symbol], 1)))

    gc.collect()
    df.fillna(method='ffill', inplace=True)
    # df.fillna(method='bfill', inplace=True)  
    for symbol in list_cryptocurrencies:
        df.drop(columns=[f'open_{symbol}', f'high_{symbol}', f'low_{symbol}', f'volume_{symbol}'], inplace=True)
    gc.collect()  
    df = df[df['timestamp'] >= limit] 
    df.drop_duplicates(subset=['timestamp'], inplace=True)
    return df   

In [129]:
def getFeaturesVer2(df, limit):
    lags_normal = [8, 20, 120]
    lags_ema = [15]
    lags_rsi = [8, 14]
    lags_stoch_osci = [10, 25]
    roc_window = [8, 15]
    williams_percentage_r = [10, 20]
    lags_wbands = [10, 50]
    for id in range(10):        
        symbol = list_cryptocurrencies[id] 
        for lag in lags_normal:
            df[f'log_close/mean_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.append(np.convolve( np.array(df[f'close_{symbol}']), np.ones(lag)/lag, mode="valid"), np.ones(lag-1)), lag-1)  )
            df[f'log_return_{lag}_{symbol}'] = np.log( np.array(df[f'close_{symbol}']) /  np.roll(np.array(df[f'close_{symbol}']), lag)  )
    for lag in lags_normal:
        df[f'mean_close/mean_{lag}'] =  np.mean(df.iloc[:,df.columns.str.startswith(f'log_close/mean_{lag}_')], axis=1)
        df[f'mean_log_returns_{lag}'] = np.mean(df.iloc[:,df.columns.str.startswith(f'log_return_{lag}_')], axis=1)
        for id in range(10):
            symbol = list_cryptocurrencies[id] 
            df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
            df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
    # apply TA indicators: EMA, RSI, Bollinger Bands, Stochastic Oscillator
    for id in range(10):
        symbol = list_cryptocurrencies[id]
        for lag in lags_ema:
            # df[f'ema_close_{lag}_{symbol}'] = ta.trend.ema_indicator(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'ema_close_{lag}_{symbol}'] = exponential_moving_average(df[f'close_{symbol}'], span=lag)
        for lag in lags_rsi:
            # df[f'rsi_close_{lag}_{symbol}'] = ta.momentum.rsi(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'rsi_close_{lag}_{symbol}'] = relative_strength_index(df[f'close_{symbol}'], window=lag)
        for lag in lags_stoch_osci:
            # df[f'stoch_oscil_close_{lag}_{symbol}'] = ta.momentum.stoch(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'stoch_oscil_close_{lag}_{symbol}'] = stochastic_oscillator(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag)
        for lag in roc_window:
            # df[f'roc_close_{lag}_{symbol}'] = ta.momentum.roc(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'roc_close_{lag}_{symbol}'] = rate_of_change(df[f'close_{symbol}'], window=lag)
        for lag in williams_percentage_r:
            # df[f'williams_close_{lag}_{symbol}'] = ta.momentum.williams_r(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], lbp=lag, fillna=False)  
            df[f'williams_close_{lag}_{symbol}'] = williams_r(df[f'high_{symbol}'], df[f'low_{symbol}'], df[f'close_{symbol}'], window=lag)
        for lag in lags_wbands:
            # df[f'wbbands_close_{lag}_{symbol}'] = ta.volatility.bollinger_wband(df[f'close_{symbol}'], window=lag, fillna=False)
            df[f'wbbands_close_{lag}_{symbol}'] = bollinger_wband(df[f'close_{symbol}'], window=lag) 
        # Alpha#12: (sign(delta(volume, 1)) * (-1 * delta(close, 1)))
        df['alpha12_'+symbol] = np.sign(delta(df['volume_'+symbol], 1)) * (-1 * delta(df['close_'+symbol], 1))
        # Alpha#41: (((high * low)^0.5) - close)
        df['alpha41_'+symbol] = ((df['high_'+symbol] * df['low_'+symbol])**0.5) - df['close_'+symbol]
        # Alpha#53: (-1 * delta((((close - low) - (high - close)) / (close - low + 0.0000001)), 9))
        df['alpha53_'+symbol] = -1 * delta((((df['close_'+symbol] - df['low_'+symbol]) - (df['high_'+symbol] - df['close_'+symbol])) / (df['close_'+symbol] - df['low_'+symbol] + 0.0000001)), 9)
        # Alpha#6: (-1 * correlation(open, volume, 10)) 
        df['alpha6_'+symbol] = -1 * correlation(df['open_'+symbol], df['volume_'+symbol], 10)
        # Alpha#9: ((0 < ts_min(delta(close, 1), 5)) ? delta(close, 1) : ((ts_max(delta(close, 1), 5) < 0) ? delta(close, 1) : (-1 * delta(close, 1)))) 
        df['alpha9_'+symbol] = np.where(0 < ts_min(delta(df['close_'+symbol], 1), 5), delta(df['close_'+symbol], 1), np.where(ts_max(delta(df['close_'+symbol], 1), 5) < 0, delta(df['close_'+symbol], 1), -1 * delta(df['close_'+symbol], 1)))
    gc.collect()  
    # df.dropna(inplace=True)
    df.fillna(method='ffill', inplace=True)
    # df.fillna(method='bfill', inplace=True)
    for symbol in list_cryptocurrencies:
        df.drop(columns=[f'high_{symbol}', f'low_{symbol}', f'open_{symbol}', f'volume_{symbol}'], inplace=True)
    gc.collect()  
    df = df[df['timestamp'] >= limit] 
    df.drop_duplicates(subset=['timestamp'], inplace=True)
    return df   

In [130]:
combined_df1 = getFeaturesVer1(combined_df1, lower_limit_test)
print(combined_df1.shape)
combined_df2 = getFeaturesVer2(combined_df2, lower_limit_test)
print(combined_df2.shape)

C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\2746239995.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )


C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\2746239995.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_return_{lag}-mean_log_returns_{lag}_{symbol}'] = np.array( df[f'log_return_{lag}_{symbol}']) - np.array( df[f'mean_log_returns_{lag}'] )
C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\2746239995.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_close/mean_{lag}-mean_close/mean_{lag}_{symbol}'] = np.array( df[f'log_close/mean_{lag}_{symbol}']) - np.array( df[f'mean_close/mean_{lag}']  )
C:\

(14401, 409)


C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\3820758813.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['alpha6_'+symbol] = -1 * correlation(df['open_'+symbol], df['volume_'+symbol], 10)
C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\3820758813.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['alpha9_'+symbol] = np.where(0 < ts_min(delta(df['close_'+symbol], 1), 5), delta(df['close_'+symbol], 1), np.where(ts_max(delta(df['close_'+symbol], 1), 5) < 0, delta(df['close_'+symbol], 1), -1 * delta(df['close_

(14401, 297)


C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\3820758813.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['timestamp'], inplace=True)


In [131]:
num_models = 12

In [132]:
def correlation(a, b):    
    a = np.ravel(a)
    b = np.ravel(b)

    len_data = len(a)
    mean_a = np.mean(a)
    mean_b = np.mean(b)
    var_a = np.var(a, ddof=0)  
    var_b = np.var(b, ddof=0)  
    
    cov = np.sum((a - mean_a) * (b - mean_b)) / len_data
    corr = cov / np.sqrt(var_a * var_b)

    return corr

In [133]:
x = 4

In [134]:
def predictForEachAssetVer1(df):
    preds = []
    output = pd.DataFrame()
    for id, symbol in enumerate(list_cryptocurrencies):
        
        pred_asset = []
        symbol = list_cryptocurrencies[id]
        columns = [col for col in df.columns if symbol in col] + ['timestamp']
        df_asset = df[columns]
        
        #load models
        models = []
        for fold in range(12):
            model = pickle.load(open(os.path.join(MODELS_DIR, f'{symbol}_fold{fold}_1.pkl'), 'rb'))
            models.append(model)
        #predict
        df_asset = df_asset.fillna(method='ffill')
        df_asset = df_asset.drop(columns=['timestamp']) 
        for model in models: 
            pred_fold = model.predict(df_asset)
            pred_asset.append(pred_fold)
        
        pred = np.mean(pred_asset, axis=0)
        del models
        preds.append(pred)
        output_symbol = pd.DataFrame({'timestamp': df['timestamp'], 'symbol': symbol, 'log_return': pred})
        output_symbol.fillna(method='ffill', inplace=True)
        output = pd.concat([output, output_symbol])
        gc.collect()
    return output

In [135]:
def predictForEachAssetVer2(df):
    preds = []
    output = pd.DataFrame()
    for id, symbol in enumerate(list_cryptocurrencies):
        
        pred_asset = []
        columns = [col for col in df.columns if symbol in col] + ['timestamp']
        df_asset = df[columns]
        
        #load models
        models = []
        for fold in range(12):
            model = pickle.load(open(os.path.join(MODELS_DIR, f'{symbol}_fold{fold}_2.pkl'), 'rb'))
            models.append(model)
        #predict
        df_asset = df_asset.fillna(method='ffill')
        df_asset = df_asset.drop(columns=['timestamp']) 
        for model in models: 
            pred_fold = model.predict(df_asset)
            pred_asset.append(pred_fold)
        
        pred = np.mean(pred_asset, axis=0)
        del models
        preds.append(pred)
        output_symbol = pd.DataFrame({'timestamp': df['timestamp'], 'symbol': symbol, 'log_return': pred})
        output_symbol.fillna(method='ffill', inplace=True)
        output = pd.concat([output, output_symbol])
        gc.collect()
    return output

In [136]:
output1 = predictForEachAssetVer1(combined_df1)
print(output1.shape)
output2 = predictForEachAssetVer2(combined_df2)
print(output2.shape)

C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\1455648859.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset.fillna(method='ffill')


C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\1455648859.py:27: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  output_symbol.fillna(method='ffill', inplace=True)
C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\1455648859.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset.fillna(method='ffill')
C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\1455648859.py:27: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  output_symbol.fillna(method='ffill', inplace=True)
C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\1455648859.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset.fil

(144010, 3)


C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\485279475.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  output_symbol.fillna(method='ffill', inplace=True)
C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\485279475.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset.fillna(method='ffill')
C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\485279475.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  output_symbol.fillna(method='ffill', inplace=True)
C:\Users\e0817820\AppData\Local\Temp\ipykernel_2288\485279475.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_asset = df_asset.fillna(

(144010, 3)


In [137]:
idx = output1[['timestamp', 'symbol']]
log_return1 = output1[['log_return']]
log_return2 = output2[['log_return']]
k = 0.5
corr_list = []
# sample = pd.read_csv(r'C:\Users\e0817820\Desktop\tokka\data\raw\submission.csv')
sample = pd.read_csv(TEST_CSV)
log_return_mean = (k*log_return1 + (1-k)*log_return2)
output = pd.concat([idx, log_return_mean], axis=1)
df_test = pd.read_csv(TEST_CSV).drop(columns=['log_return'])
output = df_test.merge(output, on=['timestamp', 'symbol'], how='left')[['id', 'log_return', 'timestamp', 'symbol']]
for id, symbol in enumerate(list_cryptocurrencies):
    pred = output[output['symbol'] == symbol][['log_return']]
    real = sample[output['symbol'] == symbol][['log_return']]
    corr_list.append(correlation(pred, real))
print(f"Corr score: {np.mean(corr_list)}")
output = output[['id', 'log_return']]
output.fillna(0)
output.to_csv(r'C:\Users\e0817820\Desktop\tokka\data\raw\submission.csv', index=False)

Corr score: 0.04697997032315761


In [138]:
print(output.shape)

(144010, 2)
